# Problema
Un sujeto de 80kg con sus facultades mentales alteradas decide pagar para tirarse de un puente atado desde los pies con una soga elastica. La altura del puente es de 20 metros y la longitud natural de la soga son 8 metros. A medida que el sujeto cae, un estudiante de la materia lo filma y puede medir con la camara la altura a cada instante de tiempo. Los datos obtenidos se pueden descargar de ......
1. Hacer el diagrama de cuerpo aislado una vez que la cuerda se empieza a estirar.
2. ¿Se puede calcular la fuerza resultante a la que esta sometido el sujeto en todo tiempo on los datos medidos? De ser posible calcularla. Si no se puede, aclarar que dato hace falta y cómo lo harian
3. Calcular el trabajo de la fuerza que hace la cuerda desde que la cuerda empieza a estirarse hasta que ya no se estira mas.
4. Graficar la energia potencial gravitatoria, potencial elastica, cinetica y energia mecanica en cada instante de la tabla.

In [4]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [105]:
ho = 20 # Altura inicial
k = 200 # Constante elastica
ln = 8 # Longitud natural
vo = 0 # Velocidad inicial
g = -9.8 # gravedad
m = 80
step = 0.03
noise_amp = 0.2

In [113]:
def caida_libre(vo,yo,to_,tf):
    t = np.arange(to_,tf,step)
    y = yo + vo*t + (g*t**2)/2
    #v = vo + g*t
    caida_df = pd.DataFrame({'t':t,
                             'y': y,
                             
                             #'a': g
                             })
    caida_df['y_noisy'] = caida_df.y + np.random.normal(loc=0, scale = noise_amp, size=len(caida_df))
    caida_df['v'] = caida_df.y.diff(-1)/caida_df.t.diff(-1)
    
    return caida_df[caida_df.y>12] # Si la Y es menor a 12 ya entra en juego la cuerda

def bungee(vo_,t,yo_):
    bungee_df = pd.DataFrame()
    hn = ho-ln
    to_ = t.copy()
    wo = (k/m)**0.5
    b = vo_/wo
    a = 12
    y = yo_ + step*vo_
    t = t+step
    while y <= 12.3:
        #y = 12 - m*g/k  + b*np.sin(wo*(t-to_) + (-m*g/k)*np.cos(wo*(t-to_)))
        y = (yo_-m*g/k) + b*np.sin(wo*(t-to_)) + (g*m/k)*np.cos(wo*(t-to_)) 
        v = -b*wo*np.cos(wo*t) - wo*(-m*g/k)*np.sin(wo*t) + vo_
        a = -b*wo*wo*np.sin(wo*t)
        temp_df = pd.DataFrame({'t':[t],
                                'y': [y],
                                #'v': [v],
                                #'a': [a]
                                })

        bungee_df = pd.concat([bungee_df,temp_df],ignore_index = True)
        t = t+step
    bungee_df['y_noisy'] = bungee_df.y + np.random.normal(loc=0, scale  = noise_amp, size=len(bungee_df))
    bungee_df['v'] = bungee_df.y.diff(-1)/bungee_df.t.diff(-1)        
    return bungee_df
        

df = pd.DataFrame({'t':[],
                   'y':[],
                   'v':[]})

caida_df = caida_libre(vo,ho,0,5)
caida_df = pd.concat([caida_df,bungee(caida_df.v.iloc[-1],caida_df.t.iloc[-1],caida_df.y.iloc[-1])],ignore_index = True)

fig = go.Figure()
fig.add_traces(go.Scatter(x = caida_df.t,y = caida_df.y_noisy,mode = 'markers+lines'))
fig.show()

In [115]:
fig = go.Figure()
fig.add_traces(go.Scatter(x = caida_df.t,y = caida_df.v,mode = 'markers+lines'))
fig.show()

In [116]:
fig = go.Figure()
fig.add_traces(go.Scatter(x = caida_df.t,y = caida_df.v.diff(-1)/caida_df.t.diff(-1),mode = 'markers+lines'))
fig.show()